In [1]:
"""
Task 17 – LangGraph Day 5
Supervisor Agent & Swarm
LLM: Ollama (llama3)
"""

from typing import TypedDict

from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


# -------------------------------------------------
# Define State
# -------------------------------------------------
class SwarmState(TypedDict):
    query: str
    agent_type: str
    response: str


# -------------------------------------------------
# LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)


# -------------------------------------------------
# Supervisor Prompt
# -------------------------------------------------
supervisor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a supervisor agent. Decide which agent should handle "
            "the user query.\n\n"
            "Available agents:\n"
            "- math_agent (math questions)\n"
            "- code_agent (programming questions)\n"
            "- general_agent (general questions)\n\n"
            "Respond with ONLY the agent name."
        ),
        ("human", "{query}")
    ]
)


# -------------------------------------------------
# Supervisor Node
# -------------------------------------------------
def supervisor(state: SwarmState) -> SwarmState:
    chain = supervisor_prompt | llm
    agent = chain.invoke({"query": state["query"]}).content.strip().lower()

    return {
        **state,
        "agent_type": agent
    }


# -------------------------------------------------
# Worker Agents
# -------------------------------------------------
def math_agent(state: SwarmState) -> SwarmState:
    try:
        # Very simple evaluation (basic demo only)
        expression = state["query"].replace("What is", "").replace("?", "").strip()
        result = eval(expression)

        return {
            **state,
            "response": f"The answer is {result}."
        }
    except Exception:
        return {
            **state,
            "response": "I could not evaluate the math expression."
        }



def code_agent(state: SwarmState) -> SwarmState:
    return {
        **state,
        "response": (
            "This is a programming-related question. "
            "Here is a simple code explanation."
        )
    }


def general_agent(state: SwarmState) -> SwarmState:
    return {
        **state,
        "response": (
            "This is a general query. "
            "Here is a helpful response."
        )
    }


# -------------------------------------------------
# Routing Logic
# -------------------------------------------------
def route_to_agent(state: SwarmState) -> str:
    if state["agent_type"] == "math_agent":
        return "math_agent"
    if state["agent_type"] == "code_agent":
        return "code_agent"
    return "general_agent"


# -------------------------------------------------
# Build LangGraph
# -------------------------------------------------
graph = StateGraph(SwarmState)

graph.add_node("supervisor", supervisor)
graph.add_node("math_agent", math_agent)
graph.add_node("code_agent", code_agent)
graph.add_node("general_agent", general_agent)

graph.set_entry_point("supervisor")

graph.add_conditional_edges(
    "supervisor",
    route_to_agent,
    {
        "math_agent": "math_agent",
        "code_agent": "code_agent",
        "general_agent": "general_agent"
    }
)

graph.add_edge("math_agent", END)
graph.add_edge("code_agent", END)
graph.add_edge("general_agent", END)

app = graph.compile()


# -------------------------------------------------
# Run
# -------------------------------------------------
if __name__ == "__main__":
    queries = [
        "What is 10 + 20?",
        "Write a simple Python function",
        "What is artificial intelligence?"
    ]

    for q in queries:
        result = app.invoke({"query": q})
        print("\nUser Query:", q)
        print("Selected Agent:", result["agent_type"])
        print("Response:", result["response"])



User Query: What is 10 + 20?
Selected Agent: math_agent
Response: The answer is 30.

User Query: Write a simple Python function
Selected Agent: code_agent
Response: This is a programming-related question. Here is a simple code explanation.

User Query: What is artificial intelligence?
Selected Agent: general_agent
Response: This is a general query. Here is a helpful response.
